In [3]:
import os
import pandas as pd
import re
import numpy as np
import netCDF4 as nc
from PIL import Image
import datetime 
import imageio
import matplotlib.pyplot as plt
import math
import xarray as xr
import glob

# Load Site Data

In [2]:
sites_df = pd.read_csv('~data/site.csv')
sites = [[row['label'], row['EUBWID'], row['lat'], row['long']] for index, row in sites_df.iterrows()]
sites[:10]

[['Ainsdale', 'ukd5300-41300', 53.60792741, -3.063987798],
 ['Allonby', 'ukd1101-46100', 54.76803625, -3.434625292],
 ['Allonby South', 'ukd1101-46000', 54.74396018, -3.456453603],
 ['Amble Links', 'ukc2101-04250', 55.32565184, -1.552400659],
 ['Anderby', 'ukf3102-09400', 53.25977424, 0.325519541],
 ["Anstey's Cove (Torquay)", 'ukk4200-23501', 50.47184615, -3.501089731],
 ['Babbacombe', 'ukk4200-23400', 50.47907733, -3.509409069],
 ['Bamburgh Castle', 'ukc2102-03700', 55.61096248, -1.707903477],
 ['Bantham', 'ukk4305-25700', 50.2789762, -3.878799945],
 ['Beachlands Central', 'ukj3307-16350', 50.78359334, -0.989275631]]

# Load single NetCDF dataset for the lat and lon grid, and snap sites to grid

In [53]:
ds = nc.Dataset("~data/cmems_obs-oc_atl_bgc-plankton_my_l3-multi-1km_P1D_1688050384618.nc")
lats = ds['lat'][:].data
lons = ds['lon'][:].data

In [99]:
ds

<xarray.Dataset>
Dimensions:  (time: 1096, lat: 15, lon: 15)
Coordinates:
  * lon      (lon) float32 1.661 1.672 1.682 1.693 ... 1.776 1.786 1.797 1.807
  * time     (time) datetime64[ns] 2020-01-01 2020-01-02 ... 2022-12-31
  * lat      (lat) float32 52.64 52.63 52.62 52.61 ... 52.53 52.52 52.51 52.49
Data variables:
    BBP      (time, lat, lon) float32 nan nan nan nan nan ... nan nan nan nan
    CDM      (time, lat, lon) float32 nan nan nan nan nan ... nan nan nan nan
Attributes: (12/92)
    FROM_ORIGINAL_FILE__lon_step:                     0.010416667
    keywords:                                         EARTH SCIENCE > OCEANS ...
    cmems_product_id:                                 OCEANCOLOUR_ATL_BGC_L3_...
    FROM_ORIGINAL_FILE__lat_step:                     0.010416667
    grid_mapping:                                     Equirectangular
    title:                                            cmems_obs-oc_atl_bgc-op...
    ...                                               ...
    product_name:                                     20230621_cmems_obs-oc_a...
    contact:                                          servicedesk.cmems@acri-...
    FROM_ORIGINAL_FILE__westernmost_longitude:        -46.0
    geospatial_vertical_positive:                     up
    date_created:                                     2023-06-29T05:05:18Z
    _CoordSysBuilder:                                 ucar.nc2.dataset.conv.C...

In [54]:
len(lats),len(lons)

(609, 814)

In [81]:
%%time
sites_grid = []
for site in sites:
    lat_orig = site[2]
    lon_orig = site[3]
    
    j = 7 # this returns 15 x 15 grid with site at the center
    
    lat_grid = lats[np.argmin(np.abs(lats - lat_orig))]
    lon_grid = lons[np.argmin(np.abs(lons - lon_orig))]
    
    lat_grid_left = lats[np.argmin(np.abs(lats - lat_orig))-j]
    lon_grid_left = lons[np.argmin(np.abs(lons - lon_orig))-j]
    
    lat_grid_right = lats[np.argmin(np.abs(lats - lat_orig))+j]
    lon_grid_right = lons[np.argmin(np.abs(lons - lon_orig))+j]
    
    sites_grid.append([site[0], site[1], lat_orig, lon_orig, lat_grid, lon_grid, 
                       lat_grid_left, lon_grid_left, lat_grid_right, lon_grid_right])
    
sites_grid[:2]

CPU times: user 19 ms, sys: 5.96 ms, total: 25 ms
Wall time: 25.9 ms


[['Ainsdale',
  'ukd5300-41300',
  53.60792741,
  -3.063987798,
  53.609375,
  -3.067707,
  53.682293,
  -3.1406238,
  53.536457,
  -2.9947903],
 ['Allonby',
  'ukd1101-46100',
  54.76803625,
  -3.434625292,
  54.765625,
  -3.4322903,
  54.838543,
  -3.505207,
  54.692707,
  -3.3593738]]

# Loop through all sites

## Download .nc files

### plankton-multi-1km

In [85]:
%%time
# general
username = "slai"
password = "Capstone2023"
motu_server = "https://my.cmems-du.eu/motu-web/Motu"
service_id = "OCEANCOLOUR_ATL_BGC_L3_MY_009_113-TDS"

output_directories = ["/Users/yuhanwang/Desktop/CourseMaterial/Capstone/capstone-project-sewage-pollution/~data/" + x 
               for x in ["plankton-multi-1km","optics-multi-1km","transp-multi-1km"]]
product_ids = ["cmems_obs-oc_atl_bgc-plankton_my_l3-multi-1km_P1D", 
               "cmems_obs-oc_atl_bgc-optics_my_l3-multi-1km_P1D",
               "cmems_obs-oc_atl_bgc-transp_my_l3-multi-1km_P1D"]


# Loop through sites_grid (lists of list, with each list corresponding to a site and its corresponding lon/lat markings)
for i in list(range(3)):
    for site in sites_grid:
        # site info
        site_id = site[1]
        lat_grid_max = site[6]
        lat_grid_min = site[8]
        lon_grid_max = site[9]
        lon_grid_min = site[7]

        # query specifications
        output_filename = f"{site_id}.nc"
        date_min = '"2020-01-01 00:00:00"'
        date_max = '"2022-12-31 23:59:59"'
    
        queries = [('python -m motuclient --motu '+ motu_server + ' --service-id '+ service_id + ' --product-id ' + product_ids[i] + 
                    f' --longitude-min {lon_grid_min} --longitude-max {lon_grid_max} --latitude-min {lat_grid_min} --latitude-max {lat_grid_max} ' +  
                    f' --date-min {date_min} --date-max {date_max}' + var +
                    ' --out-dir ' + output_directories[i] + ' --out-name ' + output_filename + ' --user ' + username + ' --pwd ' + password)
                   for var in [' --variable CHL ',' --variable BBP --variable CDM',' --variable KD490 --variable SPM --variable ZSD']]
        os.system(queries[i])

2023-06-29 22:37:49.004 [ INFO] Asynchronous mode set
2023-06-29 22:37:49.004 [ INFO] Authenticating user slai for service https://my.cmems-du.eu/motu-web/Motu
2023-06-29 22:37:50.204 [ INFO] Requesting file to download (this can take a while)...
2023-06-29 22:37:55.710 [ INFO] Authenticating user slai for service https://my.cmems-du.eu/motu-web/Motu
2023-06-29 22:38:04.867 [ INFO] The product is ready for download
2023-06-29 22:38:04.868 [ INFO] Downloading file (this can take a while)...
2023-06-29 22:38:05.095 [ INFO] File type: application/x-netcdf
2023-06-29 22:38:05.097 [ INFO] File size: 998.6 kB (998612 B)
2023-06-29 22:38:05.098 [ INFO] Downloading file /Users/yuhanwang/Desktop/CourseMaterial/Capstone/capstone-project-sewage-pollution/~data/plankton-multi-1km/ukd5300-41300.nc
2023-06-29 22:38:05.130 [ INFO] - 998.6 kB (6.6%)
2023-06-29 22:38:05.166 [ INFO] - 998.6 kB (13.1%)
2023-06-29 22:38:05.169 [ INFO] - 998.6 kB (19.7%)
2023-06-29 22:38:05.206 [ INFO] - 998.6 kB (26.3%)
2

2023-06-29 22:39:23.964 [ INFO] - 998.6 kB (6.6%)
2023-06-29 22:39:23.987 [ INFO] - 998.6 kB (13.1%)
2023-06-29 22:39:23.991 [ INFO] - 998.6 kB (19.7%)
2023-06-29 22:39:24.016 [ INFO] - 998.6 kB (26.3%)
2023-06-29 22:39:24.018 [ INFO] - 998.6 kB (32.8%)
2023-06-29 22:39:24.020 [ INFO] - 998.6 kB (39.4%)
2023-06-29 22:39:24.039 [ INFO] - 998.6 kB (45.9%)
2023-06-29 22:39:24.043 [ INFO] - 998.6 kB (52.5%)
2023-06-29 22:39:24.045 [ INFO] - 998.6 kB (59.1%)
2023-06-29 22:39:24.046 [ INFO] - 998.6 kB (65.6%)
2023-06-29 22:39:24.047 [ INFO] - 998.6 kB (72.2%)
2023-06-29 22:39:24.048 [ INFO] - 998.6 kB (78.8%)
2023-06-29 22:39:24.063 [ INFO] - 998.6 kB (85.3%)
2023-06-29 22:39:24.066 [ INFO] - 998.6 kB (91.9%)
2023-06-29 22:39:24.068 [ INFO] - 998.6 kB (98.4%)
2023-06-29 22:39:24.071 [ INFO] - 998.6 kB (100.0%)
2023-06-29 22:39:24.072 [ INFO] Processing  time : 0:00:30.715641
2023-06-29 22:39:24.072 [ INFO] Downloading time : 0:00:00.131842
2023-06-29 22:39:24.072 [ INFO] Total time       : 0

2023-06-29 22:40:28.330 [ INFO] Asynchronous mode set
2023-06-29 22:40:28.330 [ INFO] Authenticating user slai for service https://my.cmems-du.eu/motu-web/Motu
2023-06-29 22:40:28.975 [ INFO] Requesting file to download (this can take a while)...
2023-06-29 22:40:34.487 [ INFO] Authenticating user slai for service https://my.cmems-du.eu/motu-web/Motu
2023-06-29 22:40:42.530 [ INFO] The product is ready for download
2023-06-29 22:40:42.531 [ INFO] Downloading file (this can take a while)...
2023-06-29 22:40:42.685 [ INFO] File type: application/x-netcdf
2023-06-29 22:40:42.688 [ INFO] File size: 998.6 kB (998612 B)
2023-06-29 22:40:42.689 [ INFO] Downloading file /Users/yuhanwang/Desktop/CourseMaterial/Capstone/capstone-project-sewage-pollution/~data/plankton-multi-1km/ukj3307-16350.nc
2023-06-29 22:40:42.737 [ INFO] - 998.6 kB (6.6%)
2023-06-29 22:40:42.765 [ INFO] - 998.6 kB (13.1%)
2023-06-29 22:40:42.766 [ INFO] - 998.6 kB (19.7%)
2023-06-29 22:40:42.789 [ INFO] - 998.6 kB (26.3%)
2

2023-06-29 22:41:39.179 [ INFO] - 998.6 kB (45.9%)
2023-06-29 22:41:39.182 [ INFO] - 998.6 kB (52.5%)
2023-06-29 22:41:39.183 [ INFO] - 998.6 kB (59.1%)
2023-06-29 22:41:39.188 [ INFO] - 998.6 kB (65.6%)
2023-06-29 22:41:39.191 [ INFO] - 998.6 kB (72.2%)
2023-06-29 22:41:39.193 [ INFO] - 998.6 kB (78.8%)
2023-06-29 22:41:39.194 [ INFO] - 998.6 kB (85.3%)
2023-06-29 22:41:39.229 [ INFO] - 998.6 kB (91.9%)
2023-06-29 22:41:39.234 [ INFO] - 998.6 kB (98.4%)
2023-06-29 22:41:39.236 [ INFO] - 998.6 kB (100.0%)
2023-06-29 22:41:39.236 [ INFO] Processing  time : 0:00:14.257347
2023-06-29 22:41:39.237 [ INFO] Downloading time : 0:00:00.277174
2023-06-29 22:41:39.237 [ INFO] Total time       : 0:00:14.534521
2023-06-29 22:41:39.237 [ INFO] Download rate    : 1.9 MB/s
2023-06-29 22:41:39.238 [ INFO] Done
2023-06-29 22:41:39.545 [ INFO] Asynchronous mode set
2023-06-29 22:41:39.545 [ INFO] Authenticating user slai for service https://my.cmems-du.eu/motu-web/Motu
2023-06-29 22:41:40.586 [ INFO] Re

2023-06-29 22:42:44.271 [ INFO] Asynchronous mode set
2023-06-29 22:42:44.271 [ INFO] Authenticating user slai for service https://my.cmems-du.eu/motu-web/Motu
2023-06-29 22:42:45.823 [ INFO] Requesting file to download (this can take a while)...
2023-06-29 22:42:51.434 [ INFO] Authenticating user slai for service https://my.cmems-du.eu/motu-web/Motu
2023-06-29 22:42:58.733 [ INFO] The product is ready for download
2023-06-29 22:42:58.734 [ INFO] Downloading file (this can take a while)...
2023-06-29 22:42:58.983 [ INFO] File type: application/x-netcdf
2023-06-29 22:42:58.986 [ INFO] File size: 998.6 kB (998612 B)
2023-06-29 22:42:58.987 [ INFO] Downloading file /Users/yuhanwang/Desktop/CourseMaterial/Capstone/capstone-project-sewage-pollution/~data/plankton-multi-1km/ukk4305-25800.nc
2023-06-29 22:42:59.069 [ INFO] - 998.6 kB (6.6%)
2023-06-29 22:42:59.116 [ INFO] - 998.6 kB (13.1%)
2023-06-29 22:42:59.125 [ INFO] - 998.6 kB (19.7%)
2023-06-29 22:42:59.163 [ INFO] - 998.6 kB (26.3%)
2

...

2023-06-30 04:37:54.961 [ INFO] Asynchronous mode set
2023-06-30 04:37:54.962 [ INFO] Authenticating user slai for service https://my.cmems-du.eu/motu-web/Motu
2023-06-30 04:37:57.253 [ INFO] Requesting file to download (this can take a while)...
2023-06-30 04:38:03.629 [ INFO] Authenticating user slai for service https://my.cmems-du.eu/motu-web/Motu
2023-06-30 04:38:11.362 [ INFO] The product is ready for download
2023-06-30 04:38:11.363 [ INFO] Downloading file (this can take a while)...
2023-06-30 04:38:11.512 [ INFO] File type: application/x-netcdf
2023-06-30 04:38:11.514 [ INFO] File size: 3.0 MB (2973196 B)
2023-06-30 04:38:11.514 [ INFO] Downloading file /Users/yuhanwang/Desktop/CourseMaterial/Capstone/capstone-project-sewage-pollution/~data/transp-multi-1km/uke1200-08200.nc
2023-06-30 04:38:11.561 [ INFO] -   3.0 MB (2.2%)
2023-06-30 04:38:11.590 [ INFO] -   3.0 MB (4.4%)
2023-06-30 04:38:11.594 [ INFO] -   3.0 MB (6.6%)
2023-06-30 04:38:11.614 [ INFO] -   3.0 MB (8.8%)
2023-06

2023-06-30 04:38:40.370 [ INFO] -   3.0 MB (35.3%)
2023-06-30 04:38:40.378 [ INFO] -   3.0 MB (37.5%)
2023-06-30 04:38:40.388 [ INFO] -   3.0 MB (39.7%)
2023-06-30 04:38:40.406 [ INFO] -   3.0 MB (41.9%)
2023-06-30 04:38:40.409 [ INFO] -   3.0 MB (44.1%)
2023-06-30 04:38:40.422 [ INFO] -   3.0 MB (46.3%)
2023-06-30 04:38:40.440 [ INFO] -   3.0 MB (48.5%)
2023-06-30 04:38:40.443 [ INFO] -   3.0 MB (50.7%)
2023-06-30 04:38:40.445 [ INFO] -   3.0 MB (52.9%)
2023-06-30 04:38:40.458 [ INFO] -   3.0 MB (55.1%)
2023-06-30 04:38:40.477 [ INFO] -   3.0 MB (57.3%)
2023-06-30 04:38:40.481 [ INFO] -   3.0 MB (59.5%)
2023-06-30 04:38:40.483 [ INFO] -   3.0 MB (61.7%)
2023-06-30 04:38:40.486 [ INFO] -   3.0 MB (63.9%)
2023-06-30 04:38:40.499 [ INFO] -   3.0 MB (66.1%)
2023-06-30 04:38:40.517 [ INFO] -   3.0 MB (68.3%)
2023-06-30 04:38:40.522 [ INFO] -   3.0 MB (70.5%)
2023-06-30 04:38:40.526 [ INFO] -   3.0 MB (72.7%)
2023-06-30 04:38:40.535 [ INFO] -   3.0 MB (74.9%)
2023-06-30 04:38:40.551 [ INFO]

2023-06-30 04:39:14.279 [ INFO] Asynchronous mode set
2023-06-30 04:39:14.280 [ INFO] Authenticating user slai for service https://my.cmems-du.eu/motu-web/Motu
2023-06-30 04:39:15.674 [ INFO] Requesting file to download (this can take a while)...
2023-06-30 04:39:21.172 [ INFO] Authenticating user slai for service https://my.cmems-du.eu/motu-web/Motu
2023-06-30 04:39:29.394 [ INFO] The product is ready for download
2023-06-30 04:39:29.395 [ INFO] Downloading file (this can take a while)...
2023-06-30 04:39:29.555 [ INFO] File type: application/x-netcdf
2023-06-30 04:39:29.558 [ INFO] File size: 3.0 MB (2973196 B)
2023-06-30 04:39:29.558 [ INFO] Downloading file /Users/yuhanwang/Desktop/CourseMaterial/Capstone/capstone-project-sewage-pollution/~data/transp-multi-1km/ukd1203-45675.nc
2023-06-30 04:39:29.605 [ INFO] -   3.0 MB (2.2%)
2023-06-30 04:39:29.631 [ INFO] -   3.0 MB (4.4%)
2023-06-30 04:39:29.635 [ INFO] -   3.0 MB (6.6%)
2023-06-30 04:39:29.657 [ INFO] -   3.0 MB (8.8%)
2023-06

2023-06-30 04:40:02.899 [ INFO] -   3.0 MB (55.1%)
2023-06-30 04:40:02.912 [ INFO] -   3.0 MB (57.3%)
2023-06-30 04:40:02.916 [ INFO] -   3.0 MB (59.5%)
2023-06-30 04:40:02.917 [ INFO] -   3.0 MB (61.7%)
2023-06-30 04:40:02.918 [ INFO] -   3.0 MB (63.9%)
2023-06-30 04:40:02.931 [ INFO] -   3.0 MB (66.1%)
2023-06-30 04:40:02.944 [ INFO] -   3.0 MB (68.3%)
2023-06-30 04:40:02.946 [ INFO] -   3.0 MB (70.5%)
2023-06-30 04:40:02.948 [ INFO] -   3.0 MB (72.7%)
2023-06-30 04:40:02.954 [ INFO] -   3.0 MB (74.9%)
2023-06-30 04:40:02.966 [ INFO] -   3.0 MB (77.1%)
2023-06-30 04:40:02.972 [ INFO] -   3.0 MB (79.4%)
2023-06-30 04:40:02.975 [ INFO] -   3.0 MB (81.6%)
2023-06-30 04:40:02.977 [ INFO] -   3.0 MB (83.8%)
2023-06-30 04:40:02.985 [ INFO] -   3.0 MB (86.0%)
2023-06-30 04:40:03.001 [ INFO] -   3.0 MB (88.2%)
2023-06-30 04:40:03.003 [ INFO] -   3.0 MB (90.4%)
2023-06-30 04:40:03.004 [ INFO] -   3.0 MB (92.6%)
2023-06-30 04:40:03.008 [ INFO] -   3.0 MB (94.8%)
2023-06-30 04:40:03.012 [ INFO]

2023-06-30 04:40:36.994 [ INFO] Asynchronous mode set
2023-06-30 04:40:36.994 [ INFO] Authenticating user slai for service https://my.cmems-du.eu/motu-web/Motu
2023-06-30 04:40:38.744 [ INFO] Requesting file to download (this can take a while)...
2023-06-30 04:40:44.569 [ INFO] Authenticating user slai for service https://my.cmems-du.eu/motu-web/Motu
2023-06-30 04:40:52.977 [ INFO] The product is ready for download
2023-06-30 04:40:52.977 [ INFO] Downloading file (this can take a while)...
2023-06-30 04:40:53.200 [ INFO] File type: application/x-netcdf
2023-06-30 04:40:53.202 [ INFO] File size: 3.0 MB (2973196 B)
2023-06-30 04:40:53.203 [ INFO] Downloading file /Users/yuhanwang/Desktop/CourseMaterial/Capstone/capstone-project-sewage-pollution/~data/transp-multi-1km/ukj3400-18350.nc
2023-06-30 04:40:53.234 [ INFO] -   3.0 MB (2.2%)
2023-06-30 04:40:53.270 [ INFO] -   3.0 MB (4.4%)
2023-06-30 04:40:53.302 [ INFO] -   3.0 MB (6.6%)
2023-06-30 04:40:53.307 [ INFO] -   3.0 MB (8.8%)
2023-06

## Convert to CSV and merge

In [8]:
nc_files = glob.glob('/Users/yuhanwang/Desktop/Capstone/Capstone/capstone-project-sewage-pollution/~data/plankton-multi-1km/*.nc')
ds_list = [xr.open_dataset(file) for file in nc_files]

with open("plankton.csv", 'w') as f:
    for file in nc_files:

        ds = xr.open_dataset(file)
        df = ds.to_dataframe()

        site = file.split('/')[-1].replace(".nc", "")
        df['site'] = site

        if file == nc_files[0]:
            df.to_csv(f)
        else:
            df.to_csv(f, mode='a', header=False)

        ds.close()

In [9]:
nc_files = glob.glob('/Users/yuhanwang/Desktop/Capstone/Capstone/capstone-project-sewage-pollution/~data/transp-multi-1km/*.nc')
ds_list = [xr.open_dataset(file) for file in nc_files]

with open("transp.csv", 'w') as f:
    for file in nc_files:

        ds = xr.open_dataset(file)
        df = ds.to_dataframe()

        site = file.split('/')[-1].replace(".nc", "")
        df['site'] = site

        if file == nc_files[0]:
            df.to_csv(f)
        else:
            df.to_csv(f, mode='a', header=False)

        ds.close()

In [11]:
nc_files = glob.glob('/Users/yuhanwang/Desktop/Capstone/Capstone/capstone-project-sewage-pollution/~data/optics-multi-1km/*.nc')
ds_list = [xr.open_dataset(file) for file in nc_files]

with open("optics.csv", 'w') as f:
    for file in nc_files:

        ds = xr.open_dataset(file)
        df = ds.to_dataframe()

        site = file.split('/')[-1].replace(".nc", "")
        df['site'] = site

        if file == nc_files[0]:
            df.to_csv(f)
        else:
            df.to_csv(f, mode='a', header=False)

        ds.close()